aws rpc wrapper?


In [ ]:
var importer = require('../Core');
var assert = require('assert');
var getResult = importer.import('rpc result');

function handler(event, context, callback) {
    var body = event || {};
    try {
        if (event.body || event.queryStringParameters) {
            body = Object.assign(event.body || {}, event.queryStringParameters || {});
        }

        assert(body.function, 'something is wrong with your search ' + JSON.stringify(body));
        assert(importer.interpret(body.function).id, 'something is terribly wrong with function lookup ' + importer.interpret(body.function))
    }
    catch(e) {
        callback(e, {
            'statusCode': 500,
            'headers': { 
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            },
            'body': JSON.stringify({'Error': e.message})
        })
        return;
    }
    
    // TODO: add Eloqua Notify service entry point for retrieving temporary data?
    // TODO: parse action and call from notify service or call with posted data?
    // TODO: add an entry point for Zuora subscription callout to update single records in eloqua?
    return Promise.resolve([])
        .then(() => getResult({
            command: body['function'],
            result: importer.interpret(body['function']),
            body: body,
            circles: ['Public']
        }))
        .then(r => callback(null, {
            'statusCode': 200,
            'headers': { 
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            },
            'body': JSON.stringify(r, null, 4)
        }))
        // TODO: object assign error?
        .catch(e => callback(e, {
            'statusCode': 500,
            'headers': { 
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            },
            'body': JSON.stringify({'Error': e.message})
        }));
}

if(typeof module.exports === 'undefined') {
    module.exports = {};
}
module.exports.handler = handler;


latest s3 bucket?


In [1]:
var fs = require('fs');
var path = require('path');
var minimatch = require("minimatch")
var AWS = require('aws-sdk');
var importer = require('../Core');

// For dev purposes only
var AWS_HTTP = 'https://s3-us-west-2.amazonaws.com/selenium-bots/';
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
AWS.config.update(JSON.parse(fs.readFileSync(path.join(PROFILE_PATH, '.credentials/aws-sdk.json')).toString()));

function latestS3(match, bucket) {
    var s3 = new AWS.S3();
    return new Promise((resolve, reject) => s3.listObjects({
        Bucket: bucket || 'selenium-bots',
        Prefix: ''
    }, function (err, resp) {
        if(err) {
            return reject(err);
        }
        return resolve(resp.Contents
                       .sort((a, b) => b.LastModified.getTime() - a.LastModified.getTime())
                       .filter(s => !match || minimatch(s.Key, match))
                       .slice(0, 5)
                       .map(i => AWS_HTTP + i.Key));
    }));
}
module.exports = latestS3;

if(typeof $$ !== 'undefined') {
    $$.async();
    latestS3()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


[ 'https://s3-us-west-2.amazonaws.com/selenium-bots/screenshot-2018-1-1-13-29-11.png',
  'https://s3-us-west-2.amazonaws.com/selenium-bots/screenshot-2018-1-1-13-29-0.png',
  'https://s3-us-west-2.amazonaws.com/selenium-bots/screenshot-2018-1-1-13-28-40.png',
  'https://s3-us-west-2.amazonaws.com/selenium-bots/screenshot-2018-1-1-13-5-7.png',
  'https://s3-us-west-2.amazonaws.com/selenium-bots/screenshot-2018-1-1-13-1-27.png' ]